In [1]:
# Import dependencies
import pandas as pd
from sodapy import Socrata

# Load Data

In [2]:
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("chronicdata.cdc.gov", None)

In [15]:
# Return results as csv from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("swc5-untb", limit=2000000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

# View all columns in dataframes
pd.set_option('display.max_columns', None)  

# Display dataframe
results_df.head()

,year,stateabbr,statedesc,locationname,datasource,category,measure,data_value_unit,data_value_type,data_value,low_confidence_limit,high_confidence_limit,totalpopulation,locationid,categoryid,measureid,datavaluetypeid,short_question_text,geolocation,:@computed_region_bxsw_vy29,:@computed_region_he4y_prf8
0,2020,WY,Wyoming,Teton,BRFSS,Health Status,Physical health not good for >=14 days among a...,%,Crude prevalence,7.3,6.4,8.2,23497,56039,HLTHSTAT,PHLTH,CrdPrv,Physical Health,"{'type': 'Point', 'coordinates': [-110.426087,...",14,3126
1,2020,WY,Wyoming,Goshen,BRFSS,Health Status,Fair or poor self-rated health status among ad...,%,Crude prevalence,13.8,12.0,15.8,13235,56015,HLTHSTAT,GHLTH,CrdPrv,General Health,"{'type': 'Point', 'coordinates': [-104.3535403...",14,890
2,2020,WY,Wyoming,Laramie,BRFSS,Prevention,"Fecal occult blood test, sigmoidoscopy, or col...",%,Age-adjusted prevalence,61.6,58.5,64.6,100595,56021,PREVENT,COLON_SCREEN,AgeAdjPrv,Colorectal Cancer Screening,"{'type': 'Point', 'coordinates': [-104.660395,...",14,3119
3,2020,WY,Wyoming,Park,BRFSS,Prevention,Visits to doctor for routine checkup within th...,%,Crude prevalence,71.0,70.0,72.0,29331,56029,PREVENT,CHECKUP,CrdPrv,Annual Checkup,"{'type': 'Point', 'coordinates': [-109.5935975...",14,3122
4,2020,WY,Wyoming,Lincoln,BRFSS,Health Outcomes,Chronic obstructive pulmonary disease among ad...,%,Age-adjusted prevalence,5.9,5.1,6.8,20253,56023,HLTHOUT,COPD,AgeAdjPrv,COPD,"{'type': 'Point', 'coordinates': [-110.6829614...",14,3120


# Data Exploration

In [10]:
# View how many years worth of data we have
results_df['year'].unique()

array(['2020', '2019'], dtype=object)

In [7]:
# View disease outcomes (possible targets for machine learning)
results_df['measureid'].unique()

array(['PHLTH', 'GHLTH', 'COLON_SCREEN', 'CHECKUP', 'COPD', 'CASTHMA',
       'TEETHLOST', 'CANCER', 'DENTAL', 'STROKE', 'CHOLSCREEN',
       'CSMOKING', 'MHLTH', 'OBESITY', 'DIABETES', 'ARTHRITIS', 'ACCESS2',
       'BINGE', 'LPA', 'CHD', 'KIDNEY', 'BPHIGH', 'HIGHCHOL', 'SLEEP',
       'COREW', 'CERVICAL', 'MAMMOUSE', 'BPMED', 'DEPRESSION', 'COREM'],
      dtype=object)

In [8]:
# View contributing health factors (possible features for machine learning)
results_df['category'].unique()

array(['Health Status', 'Prevention', 'Health Outcomes',
       'Health Risk Behaviors'], dtype=object)

# Prepare Dataframe

In [24]:
# Create a list of the health outcomes which will become our features

# Add the health outcomes to a list
features_list = results_df.loc[results_df['category'].isin(['Prevention', 'Health Risk Behaviors', 'Health Status'])]['measureid'].unique()

# Create a new dataframe with only the health outcomes
features_df = results_df.loc[results_df['measureid'].isin(features_list)]

# Filter features_df where data_value_type is Age-adjusted Prevalence
features_df = features_df.loc[features_df['data_value_type'] == 'Age-adjusted prevalence']

# View the features dataframe
features_df.head()

,year,stateabbr,statedesc,locationname,datasource,category,measure,data_value_unit,data_value_type,data_value,low_confidence_limit,high_confidence_limit,totalpopulation,locationid,categoryid,measureid,datavaluetypeid,short_question_text,geolocation,:@computed_region_bxsw_vy29,:@computed_region_he4y_prf8
2,2020,WY,Wyoming,Laramie,BRFSS,Prevention,"Fecal occult blood test, sigmoidoscopy, or col...",%,Age-adjusted prevalence,61.6,58.5,64.6,100595,56021,PREVENT,COLON_SCREEN,AgeAdjPrv,Colorectal Cancer Screening,"{'type': 'Point', 'coordinates': [-104.660395,...",14,3119
8,2020,WY,Wyoming,Weston,BRFSS,Prevention,Visits to dentist or dental clinic among adult...,%,Age-adjusted prevalence,62.9,59.5,66.5,6743,56045,PREVENT,DENTAL,AgeAdjPrv,Dental Visit,"{'type': 'Point', 'coordinates': [-104.5700202...",14,894
11,2020,WY,Wyoming,Albany,BRFSS,Health Risk Behaviors,Current smoking among adults aged >=18 years,%,Age-adjusted prevalence,15.3,12.5,18.6,38950,56001,RISKBEH,CSMOKING,AgeAdjPrv,Current Smoking,"{'type': 'Point', 'coordinates': [-105.7218826...",14,3079
21,2020,WY,Wyoming,Platte,BRFSS,Health Status,Fair or poor self-rated health status among ad...,%,Age-adjusted prevalence,12.1,10.5,13.8,8578,56031,HLTHSTAT,GHLTH,AgeAdjPrv,General Health,"{'type': 'Point', 'coordinates': [-104.9539626...",14,892
23,2020,WY,Wyoming,Platte,BRFSS,Prevention,Visits to dentist or dental clinic among adult...,%,Age-adjusted prevalence,66.8,63.2,70.2,8578,56031,PREVENT,DENTAL,AgeAdjPrv,Dental Visit,"{'type': 'Point', 'coordinates': [-104.9539626...",14,892


In [11]:
# Create the COPD dataframe by filtering results_df which will be our taget
copd_df = results_df.loc[results_df["measureid"] == "COPD", :]

# Filter copd_df where data_value_type is Age-adjusted Prevalence
copd_df = copd_df.loc[copd_df["data_value_type"] == "Age-adjusted prevalence", :]

# Display dataframe and view all columns
copd_df.head(3)

,year,stateabbr,statedesc,locationname,datasource,category,measure,data_value_unit,data_value_type,data_value,low_confidence_limit,high_confidence_limit,totalpopulation,locationid,categoryid,measureid,datavaluetypeid,short_question_text,geolocation,:@computed_region_bxsw_vy29,:@computed_region_he4y_prf8
4,2020,WY,Wyoming,Lincoln,BRFSS,Health Outcomes,Chronic obstructive pulmonary disease among ad...,%,Age-adjusted prevalence,5.9,5.1,6.8,20253,56023,HLTHOUT,COPD,AgeAdjPrv,COPD,"{'type': 'Point', 'coordinates': [-110.6829614...",14,3120
15,2020,WY,Wyoming,Crook,BRFSS,Health Outcomes,Chronic obstructive pulmonary disease among ad...,%,Age-adjusted prevalence,6.1,5.1,7.0,7593,56011,HLTHOUT,COPD,AgeAdjPrv,COPD,"{'type': 'Point', 'coordinates': [-104.5672975...",14,3083
20,2020,WY,Wyoming,Platte,BRFSS,Health Outcomes,Chronic obstructive pulmonary disease among ad...,%,Age-adjusted prevalence,6.3,5.4,7.1,8578,56031,HLTHOUT,COPD,AgeAdjPrv,COPD,"{'type': 'Point', 'coordinates': [-104.9539626...",14,892


In [26]:
# # Combine the features and target dataframes into one dataframe on statedesc and locationname columns
# combined_df = pd.merge(features_df, copd_df, on=['statedesc', 'locationname'], how='inner')

# # Display dataframe and view all columns
# combined_df.head(3)


,year_x,stateabbr_x,statedesc,locationname,datasource_x,category_x,measure_x,data_value_unit_x,data_value_type_x,data_value_x,low_confidence_limit_x,high_confidence_limit_x,totalpopulation_x,locationid_x,categoryid_x,measureid_x,datavaluetypeid_x,short_question_text_x,geolocation_x,:@computed_region_bxsw_vy29_x,:@computed_region_he4y_prf8_x,year_y,stateabbr_y,datasource_y,category_y,measure_y,data_value_unit_y,data_value_type_y,data_value_y,low_confidence_limit_y,high_confidence_limit_y,totalpopulation_y,locationid_y,categoryid_y,measureid_y,datavaluetypeid_y,short_question_text_y,geolocation_y,:@computed_region_bxsw_vy29_y,:@computed_region_he4y_prf8_y
0,2020,WY,Wyoming,Laramie,BRFSS,Prevention,"Fecal occult blood test, sigmoidoscopy, or col...",%,Age-adjusted prevalence,61.6,58.5,64.6,100595,56021,PREVENT,COLON_SCREEN,AgeAdjPrv,Colorectal Cancer Screening,"{'type': 'Point', 'coordinates': [-104.660395,...",14,3119,2020,WY,BRFSS,Health Outcomes,Chronic obstructive pulmonary disease among ad...,%,Age-adjusted prevalence,5.7,4.9,6.4,100595,56021,HLTHOUT,COPD,AgeAdjPrv,COPD,"{'type': 'Point', 'coordinates': [-104.660395,...",14,3119
1,2020,WY,Wyoming,Laramie,BRFSS,Health Risk Behaviors,Sleeping less than 7 hours among adults aged >...,%,Age-adjusted prevalence,30.4,28.9,31.6,100595,56021,RISKBEH,SLEEP,AgeAdjPrv,Sleep <7 hours,"{'type': 'Point', 'coordinates': [-104.660395,...",14,3119,2020,WY,BRFSS,Health Outcomes,Chronic obstructive pulmonary disease among ad...,%,Age-adjusted prevalence,5.7,4.9,6.4,100595,56021,HLTHOUT,COPD,AgeAdjPrv,COPD,"{'type': 'Point', 'coordinates': [-104.660395,...",14,3119
2,2019,WY,Wyoming,Laramie,BRFSS,Prevention,Cholesterol screening among adults aged >=18 y...,%,Age-adjusted prevalence,84.1,82.7,85.4,100595,56021,PREVENT,CHOLSCREEN,AgeAdjPrv,Cholesterol Screening,"{'type': 'Point', 'coordinates': [-104.660395,...",14,3119,2020,WY,BRFSS,Health Outcomes,Chronic obstructive pulmonary disease among ad...,%,Age-adjusted prevalence,5.7,4.9,6.4,100595,56021,HLTHOUT,COPD,AgeAdjPrv,COPD,"{'type': 'Point', 'coordinates': [-104.660395,...",14,3119


In [13]:
# # Remove uninformative columns
# copd_df = copd_df.drop(["datasource", "category", ""], axis=1)

# # smokers_df = smokers_df.drop(["Data_Value_Footnote", "Year", "DataSource",
# #                            "Data_Value_Footnote_Symbol", "Data_Value_Footnote",
# #                            "CategoryID", "MeasureId", "DataValueTypeID",
# #                            "Short_Question_Text", "Category", "LocationName",
# #                            "StateAbbr", "StateDesc"], axis=1)